In [1]:
from SRS import SRS
import llm as L
from llama_cpp import Llama
import re

In [2]:
srs = SRS()
srs.add_card("fox")
print(srs.get_card("fox"))
print(srs.get_reps("fox"))
srs.review_card("fox", "good")
print(srs.get_card("fox"))
print(srs.get_reps("fox"))
srs.output()

0
1


In [3]:
def logits_processor(prev_tok_ids, next_tok_logits):
    
    # print(prev_tok_ids, next_tok_logits)

    next_tok_logits[995] -= 1000
 
    return next_tok_logits

In [4]:
llm = L.LLM('You are language chatbot. You speak using low-level, easy to comprehend words. You end every sentence with a short quiz to see if they understand')

Initializing Global LLM Instance


In [5]:
user_input = input()
resp = llm(user_input, max_tokens=200, logits_processor=[logits_processor])
resp

' I speak simple. Me say "how about that?" Now question: What is my last sentence in English? Answer: "What is my last sentence in English?" (Answer: "How about that?")'

In [6]:
#[^\w\s\-]*
output_words = re.sub("[^A-Za-z0-9 -]", "", resp)

for i in output_words:
    srs.review_card(i, 'again')
# TODO: review card instead of adding
#       'good' or 'bad'?
#       how to space words? language dictionary?
# Add Lucas's model
# add bad but not very bad
# LLM to extract incorrect words (structured output)
# LLM to break words apart betterer

In [7]:
srs.output()

None
